In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from jaxtyping import Float
from torch import Tensor


/home/ubuntu/whitebox_evals/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Ministral-8B-Instruct-2410", torch_dtype=torch.bfloat16, device_map="auto")

2025-05-23 20:17:36.377313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748031456.386842   53042 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748031456.391873   53042 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.39it/s]


In [4]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-

In [3]:
torch.manual_seed(42)
vector = torch.randn(2, 4096, device=model.device)


def orthogonalize_vectors(dirs_KD: Float[Tensor, "K d_model"]) -> Float[Tensor, "K d_model"]:
    dirs_KD = torch.nn.functional.normalize(dirs_KD, dim=1)

    # Orthogonalize the directions using QR decomposition
    # Transpose to (D, K) for QR, then transpose back
    dirs_DK = dirs_KD.T
    Q, R = torch.linalg.qr(dirs_DK, mode="reduced")
    # Q is (D, K) with orthonormal columns
    dirs_KD_ortho = Q.T  # Back to (K, D)

    # Note: If K > D or vectors are linearly dependent, some columns of Q might be zero
    # We should filter those out
    norms = torch.norm(dirs_KD_ortho, dim=1)
    valid_mask = norms > 1e-6
    dirs_KD_ortho = dirs_KD_ortho[valid_mask]

    return dirs_KD_ortho

vector = orthogonalize_vectors(vector).to(dtype=torch.float32)

dirs = {}

for i, layer in enumerate(model.model.layers):
    dirs[i] = [vector[0], vector[1]]

In [ ]:
def orthogonalize_matrix_to_directions(
    W: Float[Tensor, "d_out d_in"], 
    directions: list[Float[Tensor, "d_out"]]
) -> Float[Tensor, "d_out d_in"]:
    """
    Orthogonalize a weight matrix W so it doesn't write to specified directions.
    
    W_new = W - sum_k (r_k @ r_k.T @ W)
    
    where r_k are normalized direction vectors.
    """
    W_ortho = W.clone().to(dtype=torch.float32)
    
    # First orthogonalize the directions themselves (like in your hook)
    if len(directions) > 0:
        dirs_KD = torch.stack(directions)
        dirs_KD_ortho = orthogonalize_vectors(dirs_KD)
        
        # Project out each orthogonalized direction
        for direction in dirs_KD_ortho:
            direction = direction / (direction.norm() + 1e-8)  # Normalize
            # W_new = W - r @ r.T @ W
            projection = torch.outer(direction, direction) @ W_ortho
            W_ortho = W_ortho - projection
    
    return W_ortho.to(dtype=torch.bfloat16)


def orthogonalize_model_weights(
    model: AutoModelForCausalLM,
    layer_directions: dict[int, list[Float[Tensor, "d_model"]]]
):
    """
    Orthogonalize all residual stream writes in the model.
    
    Args:
        model: The transformer model
        layer_directions: Dict mapping layer index to list of directions
    """
    
    # 1. Orthogonalize embedding matrix (affects all layers)
    # You might want to use directions from layer 0 or average across layers
    if 0 in layer_directions:
        model.model.embed_tokens.weight.data = orthogonalize_matrix_to_directions(
            model.model.embed_tokens.weight.data.T,  # Transpose to (d_model, vocab_size)
            layer_directions[0]
        ).T  # Transpose back
    
    # 2. Orthogonalize each layer's output projections
    for layer_idx, layer in enumerate(model.model.layers):
        if layer_idx not in layer_directions:
            continue
            
        directions = layer_directions[layer_idx]
        
        # Attention output projection
        layer.self_attn.o_proj.weight.data = orthogonalize_matrix_to_directions(
            layer.self_attn.o_proj.weight.data,
            directions
        )
        
        # MLP output projection  
        layer.mlp.down_proj.weight.data = orthogonalize_matrix_to_directions(
            layer.mlp.down_proj.weight.data,
            directions
        )
    
    return model

model = orthogonalize_model_weights(model, dirs)